In [286]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [287]:
# Setting the current working directory
import os; os.chdir('/content/drive/MyDrive/AI_ML/Projects/Capstone-NLP/')

### Importing the Packages

In [288]:
# Importing the libraries
# load packages
import os
from flask import Flask, flash, request, redirect, url_for
from werkzeug.utils import secure_filename

import flask
app = flask.Flask(__name__)
from flask import Flask, render_template, request

import pickle

In [289]:
#UPLOAD_FOLDER = '/path/to/the/uploads'

UPLOAD_FOLDER = '/content/drive/MyDrive/AI_ML/Projects/Capstone-NLP/'

#ALLOWED_EXTENSIONS = {'txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'}

ALLOWED_EXTENSIONS = {'csv'}

In [290]:
!mkdir templates -p

In [291]:
#!ls

In [292]:
%%writefile templates/prediction.html

<!DOCTYPE html>
<div class="header">
  <h1>Accident Level</h1>
</div>

<!--
<div id="search-info">
  <!--<h2>There is a {{ prediction }}% chance your text is Accident Level.</h2>
</div>
-->

<div id="search-info">
  <!--<h2>There is a {{ prediction }}% chance your text is Accident Level.</h2>
</div>

<style>
.header {
  font-family: 'Proxima Novas', sans-serif;
  padding: 10px;
  text-align: center;
  background: #0077cc;
  color: #ffffff;
  font-size: 18px;
}

#search-info {
  text-align: center;
  font-family: 'Proxima Nova', sans-serif;
  color: #0077cc;
  font-size: 18px;
  font-weight: 100;
  padding: 20px;
  margin: -30px -30px -10px -30px;
}

</style>


Overwriting templates/prediction.html


In [293]:
%%writefile templates/search_page.html

<!DOCTYPE html>
<div class="header">
  <h1>Industrial-safety-and-health-analytics-database</h1>
</div>

<div id="search-info">
  <h2>Accident Severity Level predictor</h2>
</div>

<div id="search-info">
  <h3>A) Milestone-1)-Tasks</h3>
</div>

<div id="search-info">
  <h4>1) Import dataset into DataFrame</h4>
</div>

<div class="form-style-8">
  <form id="load_form" action="/load" method="POST" target="_blank" enctype=multipart/form-data>
	  <input type="submit" value="Import dataset" id="LoadButton" />
  </form>
</div>


<div id="search-info">
  <h4>2) Dataset Cleaning</h4>
</div>

	<div class="form-style-8">
	  <form id="Clean" action="/clean" method="POST" target="_blank" enctype=multipart/form-data>
		  <input type="submit" value="Clean Dataset" id="CleanButton" />
	  </form>
	</div>


<div id="search-info">
  <h4>3) Data preparation to be used for AIML model learning</h4>
</div>

<div class="form-style-8">
  <form id="prep" action="/prep" method="POST" target="_blank" enctype=multipart/form-data>
	  <input type="submit" value="Prep Dataset" id="PrepButton" />
  </form>
</div>

<div id="search-info">
	<h3>B) Milestone-2-Tasks</h3>
</div>

<div id="search-info">
  <h4>1) NLP pre processing</h4>
</div>

	<div class="form-style-8">
	  <form id="train-test" action="/nlp_preprocess" method="POST" target="_blank" enctype=multipart/form-data>
		  <input type="submit" value="NLP_Preprocess" id="NLP_Preprocess" />
	  </form>
	</div>


<div id="search-info">
  <h4>2) Design, train and test machine learning classifiers</h4>
</div>

	<div class="form-style-8">
	  <form id="ML_clf" action="/ML_clf" method="POST" target="_blank" enctype=multipart/form-data>
		  <input type="submit" value="ML_clf" id="ML_clfButton" />
	  </form>
	</div>

<div id="search-info">
  <h4>3) Design, train and test Neural networks classifiers</h4>
</div>

	<div class="form-style-8">
	  <form id="NN_clf" action="/NN_clf" method="POST" target="_blank" enctype=multipart/form-data>
		  <input type="submit" value="NN_clf" id="NN_clfButton" />
	  </form>
	</div>

<div id="search-info">
  <h4>4) Design, train and test RNN or LSTM classifiers</h4>
</div>
	<div class="form-style-8">
	  <form id="LSTM_clf" action="/LSTM_clf" method="POST" target="_blank" enctype=multipart/form-data>
		  <input type="submit" value="LSTM_clf" id="LSTM_clfButton" />
	  </form>
	</div>
<div id="search-info">
  <h4>5) Predicts with LSTM</h4>
</div>
	<div class="form-style-8">
	  <form id="search_form" action="/predicts_lstm" method="POST" target="_blank">
		<input type="text" name="text_entered" placeholder="Enter text received" />
		<input type="submit" value="Get Accident Level prediction!" id="PredictButton" />
	  </form>
</div>								

<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.11.1/jquery.min.js"></script>
<script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js"></script>

<script>

window.onload = function() {
  jQuery(document).keypress(function(e) {
    if (e.keyCode == 13) {
      e.preventDefault();
      jQuery('#SubmitButton').click();
    }
  });
}

</script>

<style>

h1{
  font-weight: bold;
  font-size: 2em;
  font-family: 'Raleway', sans-serif;
  margin: 0;
  margin-top: 1px;
  color: black;
  text-align: center;
  
}

h2{
  font-weight: bold;
  font-size: 1em;
  font-family: 'Raleway', sans-serif;
  margin: 0 auto;
  margin-top: 10px;
  width: 1000px;
  color: black;
  text-align: left;

}

h3 {
  text-align: left;
  padding-left: 3%;
  font-size: 1.0em;
  font-weight: bold;
  color: black;
  font-family: 'Raleway', sans-serif;
  background-color: lightblue
}

h4 {
  text-align: left;
  padding-left: 3%;
  font-size: 0.8em;
  font-weight: bold;
  color: black;
  font-family: 'Sniglet', monospace;
}

#search-info {
  text-align: center;
  font-family: 'Proxima Nova', sans-serif;
  color: #0077cc;
  font-size: 24px;
  font-weight: 100;
  padding: 20px;
  margin: -30px -30px -10px -30px;
}

</style>

Overwriting templates/search_page.html


In [294]:
!pip install flask-ngrok

In [295]:
#load model preprocessing
import numpy as np
import pandas as pd
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras.models
from keras.models import model_from_json

In [296]:
# # Load tokenizer for preprocessing
# with open('tokenizer.pickle', 'rb') as handle:
#     tokenizer = pickle.load(handle)

In [297]:
# # Load pre-trained model into memory
# json_file = open('model.json','r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)

In [298]:
# # Load weights into new model
# loaded_model.load_weights("model.h5")

In [299]:
def prepData(text):
    # Convert to array
    textDataArray = [text]
    
    # Convert into list with word ids
    Features = tokenizer.texts_to_sequences(textDataArray)
    Features = pad_sequences(Features, 20, padding='post')
    
    return Features

In [300]:
#loaded_model.compile(optimizer="Adam",loss='binary_crossentropy',metrics=['accuracy'])

In [301]:
from flask_ngrok import run_with_ngrok 

app = Flask(__name__) 
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
run_with_ngrok(app)   

In [302]:
#@app.route("/")
#def home():
#    return "<h1>Running Flask on Google Colab!</h1>"
#    #return render_template('prediction.html', prediction=prediction)

In [303]:
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/load', methods=['GET', 'POST'])
def load_dataset():
  data = pd.read_csv("IHMStefanini_industrial_safety_and_health_database_with_accidents_description.csv")
  pickle.dump( data, open( "raw_data.p", "wb" ) )
  html = data.to_html()
  return html
  #return "Data Uploaded Successfully in Dataframe"


@app.route('/clean', methods=['GET', 'POST'])
def data_clean():
  #data = pd.read_csv("IHMStefanini_industrial_safety_and_health_database_with_accidents_description.csv")
  data = pickle.load( open( "raw_data.p", "rb" ) )
  data.drop("Unnamed: 0", axis=1, inplace=True)
  data.rename(columns={'Data':'Date', 'Countries':'Country', 'Genre':'Gender', 'Employee or Third Party':'Employee type'}, inplace=True)
  replace_values = {'Local_01': 1, 'Local_02': 2, 'Local_03': 3, 'Local_04': 4, 'Local_05': 5, 'Local_06': 6, 'Local_07': 7, 'Local_08': 8, 'Local_09': 9, 'Local_10': 10, 'Local_11': 11, 'Local_12': 12}
  data['Local'] = data['Local'].map(replace_values)
  replace_values = {'I': 1, 'II': 2, 'III': 3, 'IV': 4, 'V': 5}
  data['Accident Level'] = data['Accident Level'].map(replace_values)
  replace_values = {'I': 1, 'II': 2, 'III': 3, 'IV': 4, 'V': 5, 'VI': 6}
  data['Potential Accident Level'] = data['Potential Accident Level'].map(replace_values)
  del replace_values
  pickle.dump( data, open( "clean_data.p", "wb" ) )
  html = data.to_html()
  #return "File Cleaned Successfully"
  return html

@app.route('/prep', methods=['GET', 'POST'])
def data_prep():
  data = pickle.load( open( "clean_data.p", "rb" ) )
  html = data.to_html()
  return html

@app.route('/Train-Test', methods=['GET', 'POST'])
def model_train_test_ML_classifiers():
  pass

# @app.route('/', methods = ['GET', 'POST'])
# def predicts(new_text):
#   pass

# @app.route('/predicts', methods = ['GET', 'POST'])
# def predicts(new_text):
#   new_text = ["During the tests of the soft starter of the engine of belt 03 the collaborator Igor moves around the site and at approximately 19:45 pm falls into a trench for electric cables, (0.90m deep), the same as was found partially discovered.", "", ""]
#   new_text = np.array(new_text, dtype=object)[:, np.newaxis]
#   with tf.Session() as session:
#     K.set_session(session)
#     session.run(tf.global_variables_initializer())
#     session.run(tf.tables_initializer())
#     model.load_weights('./model_lstm.h5')  
#     predicts = model.predict(new_text, batch_size=32)
#   html = predicts.to_html()
#   return html


#define a predict function as an endpoint 

@app.route('/', methods=['GET','POST'])
def predicts_lstm():
    
    #whenever the predict method is called, we're going
    #to input the user entered text into the model
    #and return a prediction
    
    if request.method=='POST':
        new_text = request.form.get('text_entered')
    #     with tf.Session() as session:
    #       K.set_session(session)
    #       session.run(tf.global_variables_initializer())
    #       session.run(tf.tables_initializer())
    #       model.load_weights('./model_lstm.h5')
    #       predicts = model.predict(new_text, batch_size=32)
    # html = predicts.to_html()
    # return html
        Features = prepData(new_text)
        prediction = int((np.asscalar(loaded_model.predict(Features)))*100)
        return render_template('prediction.html', prediction='prediction')
        #print(prediction.dtypes)
    
    else:
        return render_template("search_page.html")


In [304]:
if __name__ == "__main__":
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://856926baff52.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [10/Apr/2021 02:46:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2021 02:46:24] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/Apr/2021 02:46:26] "POST /load HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2021 02:46:29] "POST /clean HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2021 02:46:40] "POST /predicts_lstm HTTP/1.1" 404 -
